Restarted finetuning (Python 3.10.4)

In [1]:
import os
os.chdir("/work/pi_hzhang2_umass_edu/jnainani_umass_edu/diffusion_policy_interp/")
from sae.sae_model import SparseAutoencoder
import torch 
import pandas as pd 
from tqdm.auto import tqdm
from skvideo.io import vwrite
from IPython.display import Video
from diffusion_policy.policy.diffusion_transformer_lowdim_policy import DiffusionTransformerLowdimPolicy
from diffusion_policy.model.diffusion.transformer_for_diffusion import TransformerForDiffusion
from diffusers.schedulers.scheduling_ddpm import DDPMScheduler
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm
import collections
import random
from diffusion_policy.env.pusht.pusht_keypoints_env import PushTKeypointsEnv
import matplotlib.pyplot as plt

device = 'cuda' if torch.cuda.is_available() else 'cpu'

seed1_acts = pd.read_csv("data/activations_summary.csv")
seed2_acts = pd.read_csv("data/activations_summary_2.csv")

sae_path = "sae/results_layer4_dim2048_k64_auxk64_dead200/checkpoints/last.ckpt"
sae_weights = torch.load(sae_path)
ckpt = {}
for k in sae_weights['state_dict'].keys():
    if k.startswith('sae_model.'):
        ckpt[k.split(".")[1]] = sae_weights['state_dict'][k]
sae = SparseAutoencoder(256, 2048, 64, 64, 32, 200)
sae.load_state_dict(ckpt)
sae.to(device)

ckpt_path = "/work/pi_hzhang2_umass_edu/jnainani_umass_edu/Interp4Robotics/diffusionInterp/data/experiments/low_dim/pusht/diffusion_policy_transformer/train_2/checkpoints/latest.ckpt"
state_dict = torch.load(ckpt_path, map_location=device)
config = state_dict['cfg']
model_config = config['policy']['model']
model_config = {k: v for k, v in model_config.items() if not k.startswith('_target_')}
model = TransformerForDiffusion(**model_config)
noise_scheduler_config = config['policy']['noise_scheduler']
noise_scheduler = DDPMScheduler(**noise_scheduler_config)
policy_params = {
    'model': model,
    'noise_scheduler': noise_scheduler,
    'horizon': config['policy']['horizon'],
    'obs_dim': config['policy']['obs_dim'],
    'action_dim': config['policy']['action_dim'],
    'n_action_steps': config['policy']['n_action_steps'],
    'n_obs_steps': config['policy']['n_obs_steps'],
    'num_inference_steps': config['policy'].get('num_inference_steps', None),
    'obs_as_cond': config['policy'].get('obs_as_cond', False),
    'pred_action_steps_only': config['policy'].get('pred_action_steps_only', False),
}
policy = DiffusionTransformerLowdimPolicy(**policy_params)
policy.load_state_dict(state_dict['state_dicts']['model'])
policy.to(device)

<ipython-input-1-afa4be4c80d0>:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sae_weights = torch.load(sae_path)
<ipython-input-1-afa4be4c80d0>:38: FutureWarning: You are

DiffusionTransformerLowdimPolicy(
  (model): TransformerForDiffusion(
    (input_emb): Linear(in_features=2, out_features=256, bias=True)
    (drop): Dropout(p=0.0, inplace=False)
    (time_emb): SinusoidalPosEmb()
    (cond_obs_emb): Linear(in_features=20, out_features=256, bias=True)
    (encoder): Sequential(
      (0): Linear(in_features=256, out_features=1024, bias=True)
      (1): Mish()
      (2): Linear(in_features=1024, out_features=256, bias=True)
    )
    (decoder): TransformerDecoder(
      (layers): ModuleList(
        (0-7): 8 x TransformerDecoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
          )
          (multihead_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
          )
          (linear1): Linear(in_features=256, out_features=1024, bias=True)
          (dropout): Dropout(p=

In [2]:
feature_idx = 922 

fstats = pd.read_csv(f"sae_analysis/out/f{feature_idx}_stats.csv")
print(fstats.head())


# top_10_activations = seed2_acts.sort_values(
#     f"feature_{feature_idx}", ascending=False
# ).head(1)
# seed, step, timestep, _ = top_10_activations[['seed', 'step_idx', 'timestep', f'feature_{feature_idx}']].values[0]

  data_prefix     seed  step_idx  timestep  activation  theta_tc_deg  \
0       seed2  92400.0     136.0       1.0        2.97         -3.81   
1       seed2  92400.0     136.0       3.0        2.88         -3.81   
2       seed2  87400.0     120.0       4.0        2.83         -7.04   
3       seed2  87400.0     120.0       3.0        2.83         -7.04   
4       seed2  87400.0     120.0       6.0        2.82         -7.04   

   theta_c_deg  d_tc  ClosestK  Delta_theta_act  Delta_dist_act  d_act_curr  \
0         41.2  5.52       2.0            200.0            5.33        11.4   
1         41.2  5.52       2.0            201.0           10.40        10.6   
2         38.0  5.54       2.0           -112.0           23.40        18.6   
3         38.0  5.54       2.0           -136.0           26.00        18.3   
4         38.0  5.54       2.0            -87.9           28.70        18.9   

   d_act_target  
0          15.7  
1          15.0  
2          20.7  
3          20.5  
4 

In [3]:
fstats.iloc[0]

data_prefix          seed2
seed               92400.0
step_idx             136.0
timestep               1.0
activation            2.97
theta_tc_deg         -3.81
theta_c_deg           41.2
d_tc                  5.52
ClosestK               2.0
Delta_theta_act      200.0
Delta_dist_act        5.33
d_act_curr            11.4
d_act_target          15.7
Name: 0, dtype: object

In [4]:
list(fstats.iloc[0])

['seed2',
 92400.0,
 136.0,
 1.0,
 2.97,
 -3.81,
 41.2,
 5.52,
 2.0,
 200.0,
 5.33,
 11.4,
 15.7]

In [5]:
feature_idx = 922 

fstats = pd.read_csv(f"sae_analysis/out/f{feature_idx}_stats.csv")
print(fstats.head())
top_act_stats = list(fstats.iloc[0])
seed, step, timestep = top_act_stats[1], top_act_stats[2], top_act_stats[3]
print(seed, step, timestep)


# top_10_activations = seed2_acts.sort_values(
#     f"feature_{feature_idx}", ascending=False
# ).head(1)
# seed, step, timestep, _ = top_10_activations[['seed', 'step_idx', 'timestep', f'feature_{feature_idx}']].values[0]

  data_prefix     seed  step_idx  timestep  activation  theta_tc_deg  \
0       seed2  92400.0     136.0       1.0        2.97         -3.81   
1       seed2  92400.0     136.0       3.0        2.88         -3.81   
2       seed2  87400.0     120.0       4.0        2.83         -7.04   
3       seed2  87400.0     120.0       3.0        2.83         -7.04   
4       seed2  87400.0     120.0       6.0        2.82         -7.04   

   theta_c_deg  d_tc  ClosestK  Delta_theta_act  Delta_dist_act  d_act_curr  \
0         41.2  5.52       2.0            200.0            5.33        11.4   
1         41.2  5.52       2.0            201.0           10.40        10.6   
2         38.0  5.54       2.0           -112.0           23.40        18.6   
3         38.0  5.54       2.0           -136.0           26.00        18.3   
4         38.0  5.54       2.0            -87.9           28.70        18.9   

   d_act_target  
0          15.7  
1          15.0  
2          20.7  
3          20.5  
4 

In [6]:
target_seed = int(seed)
target_step = int(step)
obs_horizon = 2
target_timestep = int(timestep)
env = PushTKeypointsEnv()
env.seed(target_seed)
np.random.seed(target_seed)
torch.manual_seed(target_seed)
obs = env.reset()
imgs = []  #
obs_deque = collections.deque([obs[:20]] * obs_horizon, maxlen=obs_horizon)
done = False
max_steps = 200
step_idx = 0
rewards = []
policy.to('cuda')
# inference_inputs = {}
with tqdm(total=max_steps, desc=f"Seed {target_seed} Eval") as pbar:
    while not done:
        B = 1
        obs_seq = np.stack(obs_deque)
        nobs = torch.from_numpy(obs_seq).unsqueeze(0).to(device, dtype=torch.float32)

        with torch.no_grad():
            # Normalization and action inference logic from your code
            nobs = policy.normalizer['obs'].normalize(nobs)
            B, _, Do = nobs.shape
            To = policy.n_obs_steps
            T = policy.horizon
            Da = policy.action_dim
            device = policy.device
            dtype = policy.dtype
            cond = nobs[:, :To]

            shape = (B, T, Da)
            if policy.pred_action_steps_only:
                shape = (B, policy.n_action_steps, Da)
            cond_data = torch.zeros(size=shape, device=device, dtype=dtype)
            cond_mask = torch.zeros_like(cond_data, dtype=torch.bool)

            generator = None
            trajectory = torch.randn(
                size=cond_data.shape, 
                dtype=cond_data.dtype,
                device=cond_data.device,
                generator=generator)
    
            policy.noise_scheduler.set_timesteps(policy.num_inference_steps)

            
            for t in policy.noise_scheduler.timesteps:
                trajectory[cond_mask] = cond_data[cond_mask]

                model_output = policy.model(trajectory, t, cond)

                trajectory = policy.noise_scheduler.step(
                    model_output, t, trajectory, 
                    generator=generator,
                    **policy.kwargs
                ).prev_sample

            trajectory[cond_mask] = cond_data[cond_mask]
            naction_pred = trajectory[..., :Da]
            action_pred = policy.normalizer['action'].unnormalize(naction_pred)

            start = To - 1
            end = start + policy.n_action_steps
            action = action_pred[:, start:end]

            # inference_inputs[step_idx] = []
            if step_idx == target_step:
                target_obs = cond.clone().cpu().numpy()
                target_step_image = env.render(mode='rgb_array')
                target_action = action.clone().cpu().numpy()
                print(f"Step {step_idx} Image")

        naction = action.detach().to('cpu').numpy()
        action = naction[0]

        for i in range(len(action)):
            obs, reward, done, _ = env.step(action[i])
            obs_deque.append(obs[:20])
            rewards.append(reward)
            imgs.append(env.render(mode='rgb_array'))
            step_idx += 1
            pbar.update(1)
            if step_idx > max_steps:
                done = True
            if done:
                print(f"Reward for {target_seed}: ", max(rewards))
                break

from IPython.display import Video
vwrite('out/lowdim_f922_og.mp4', imgs)
Video('out/lowdim_f922_og.mp4', embed=True, width=256, height=256)

/work/pi_hzhang2_umass_edu/jnainani_umass_edu/diffusion_policy_interp/diffusion_policy/env/pusht/pymunk_keypoint_manager.py:37: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap('tab10')
Seed 92400 Eval:  68%|██████▊   | 137/200 [00:13<00:05, 10.95it/s]

Step 136 Image


Seed 92400 Eval: 201it [00:19, 10.25it/s]                         

Reward for 92400:  0.9641604420488911



/home/jnainani_umass_edu/.conda/envs/finetuning/lib/python3.10/site-packages/skvideo/io/abstract.py:514: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  self._proc.stdin.write(vid.tostring())


In [7]:
sae

SparseAutoencoder()

In [8]:
sae.W_dec

AttributeError: 'SparseAutoencoder' object has no attribute 'W_dec'

In [9]:
sae.w_dec

Parameter containing:
tensor([[-1.9589e-02,  1.3275e-04, -8.1391e-03,  ..., -2.9645e-02,
          2.1730e-02,  2.4467e-10],
        [-1.1661e-02, -3.4623e-02, -2.3108e-02,  ...,  3.6381e-03,
          8.6686e-03, -2.5538e-03],
        [-2.4968e-04,  2.4174e-02,  3.2157e-07,  ...,  3.4777e-02,
          3.2245e-02,  1.2397e-07],
        ...,
        [ 7.8762e-10,  1.8600e-18, -3.7299e-11,  ...,  4.0223e-04,
         -3.6441e-14, -1.0779e-02],
        [-3.4869e-09, -2.5094e-07,  1.6972e-14,  ...,  1.3852e-06,
         -2.7408e-02, -1.9633e-02],
        [-3.4991e-02, -2.2334e-02,  6.4922e-03,  ...,  2.4391e-02,
          4.9869e-03, -2.9570e-02]], device='cuda:0', requires_grad=True)

In [10]:
sae.w_dec.shaper

AttributeError: 'Parameter' object has no attribute 'shaper'

In [11]:
sae.w_dec.shape

torch.Size([2048, 256])

In [12]:
sae.w_dec[922].shape

torch.Size([256])

In [13]:
target_seed = int(seed)
target_step = int(step)
obs_horizon = 2
target_timestep = int(timestep)
env = PushTKeypointsEnv()
env.seed(target_seed)
np.random.seed(target_seed)
torch.manual_seed(target_seed)
obs = env.reset()
imgs = []  #
obs_deque = collections.deque([obs[:20]] * obs_horizon, maxlen=obs_horizon)
done = False
max_steps = 200
step_idx = 0
rewards = []
policy.to('cuda')

def steer_latent(layer_num, sae, steering_coefficient, latent_idx):
    def hook(module, input, output):
        output = output + steering_coefficient * sae.w_dec[latent_idx]
        return output
    return hook

layer_num = 4
handle = policy.model.decoder.layers[layer_num].register_forward_hook(steer_latent(layer_num, sae, 2, feature_idx))
try: 
    with tqdm(total=max_steps, desc=f"Seed {target_seed} Eval") as pbar:
        while not done:
            B = 1
            obs_seq = np.stack(obs_deque)
            nobs = torch.from_numpy(obs_seq).unsqueeze(0).to(device, dtype=torch.float32)

            with torch.no_grad():
                # Normalization and action inference logic from your code
                nobs = policy.normalizer['obs'].normalize(nobs)
                B, _, Do = nobs.shape
                To = policy.n_obs_steps
                T = policy.horizon
                Da = policy.action_dim
                device = policy.device
                dtype = policy.dtype
                cond = nobs[:, :To]

                shape = (B, T, Da)
                if policy.pred_action_steps_only:
                    shape = (B, policy.n_action_steps, Da)
                cond_data = torch.zeros(size=shape, device=device, dtype=dtype)
                cond_mask = torch.zeros_like(cond_data, dtype=torch.bool)

                generator = None
                trajectory = torch.randn(
                    size=cond_data.shape, 
                    dtype=cond_data.dtype,
                    device=cond_data.device,
                    generator=generator)
        
                policy.noise_scheduler.set_timesteps(policy.num_inference_steps)

                
                for t in policy.noise_scheduler.timesteps:
                    trajectory[cond_mask] = cond_data[cond_mask]

                    model_output = policy.model(trajectory, t, cond)

                    trajectory = policy.noise_scheduler.step(
                        model_output, t, trajectory, 
                        generator=generator,
                        **policy.kwargs
                    ).prev_sample

                trajectory[cond_mask] = cond_data[cond_mask]
                naction_pred = trajectory[..., :Da]
                action_pred = policy.normalizer['action'].unnormalize(naction_pred)

                start = To - 1
                end = start + policy.n_action_steps
                action = action_pred[:, start:end]

                # inference_inputs[step_idx] = []
                if step_idx == target_step:
                    target_obs = cond.clone().cpu().numpy()
                    target_step_image = env.render(mode='rgb_array')
                    target_action = action.clone().cpu().numpy()
                    # print(f"Step {step_idx} Image")

            naction = action.detach().to('cpu').numpy()
            action = naction[0]

            for i in range(len(action)):
                obs, reward, done, _ = env.step(action[i])
                obs_deque.append(obs[:20])
                rewards.append(reward)
                imgs.append(env.render(mode='rgb_array'))
                step_idx += 1
                pbar.update(1)
                if step_idx > max_steps:
                    done = True
                if done:
                    print(f"Reward for {target_seed}: ", max(rewards))
                    break
finally:
    # Remove hooks
    handle.remove()
    torch.cuda.empty_cache()
    
from IPython.display import Video
vwrite('out/lowdim_f922_steer.mp4', imgs)
Video('out/lowdim_f922_steer.mp4', embed=True, width=256, height=256)

Seed 92400 Eval: 201it [00:19, 10.49it/s]                         

Reward for 92400:  0.9906691165647455


In [14]:
target_seed = int(seed)
target_step = int(step)
obs_horizon = 2
target_timestep = int(timestep)
env = PushTKeypointsEnv()
env.seed(target_seed)
np.random.seed(target_seed)
torch.manual_seed(target_seed)
obs = env.reset()
imgs = []  #
obs_deque = collections.deque([obs[:20]] * obs_horizon, maxlen=obs_horizon)
done = False
max_steps = 200
step_idx = 0
rewards = []
policy.to('cuda')

def steer_latent(layer_num, sae, steering_coefficient, latent_idx):
    def hook(module, input, output):
        output = output + steering_coefficient * sae.w_dec[latent_idx]
        return output
    return hook

layer_num = 4
handle = policy.model.decoder.layers[layer_num].register_forward_hook(steer_latent(layer_num, sae, 20, feature_idx))
try: 
    with tqdm(total=max_steps, desc=f"Seed {target_seed} Eval") as pbar:
        while not done:
            B = 1
            obs_seq = np.stack(obs_deque)
            nobs = torch.from_numpy(obs_seq).unsqueeze(0).to(device, dtype=torch.float32)

            with torch.no_grad():
                # Normalization and action inference logic from your code
                nobs = policy.normalizer['obs'].normalize(nobs)
                B, _, Do = nobs.shape
                To = policy.n_obs_steps
                T = policy.horizon
                Da = policy.action_dim
                device = policy.device
                dtype = policy.dtype
                cond = nobs[:, :To]

                shape = (B, T, Da)
                if policy.pred_action_steps_only:
                    shape = (B, policy.n_action_steps, Da)
                cond_data = torch.zeros(size=shape, device=device, dtype=dtype)
                cond_mask = torch.zeros_like(cond_data, dtype=torch.bool)

                generator = None
                trajectory = torch.randn(
                    size=cond_data.shape, 
                    dtype=cond_data.dtype,
                    device=cond_data.device,
                    generator=generator)
        
                policy.noise_scheduler.set_timesteps(policy.num_inference_steps)

                
                for t in policy.noise_scheduler.timesteps:
                    trajectory[cond_mask] = cond_data[cond_mask]

                    model_output = policy.model(trajectory, t, cond)

                    trajectory = policy.noise_scheduler.step(
                        model_output, t, trajectory, 
                        generator=generator,
                        **policy.kwargs
                    ).prev_sample

                trajectory[cond_mask] = cond_data[cond_mask]
                naction_pred = trajectory[..., :Da]
                action_pred = policy.normalizer['action'].unnormalize(naction_pred)

                start = To - 1
                end = start + policy.n_action_steps
                action = action_pred[:, start:end]

                # inference_inputs[step_idx] = []
                if step_idx == target_step:
                    target_obs = cond.clone().cpu().numpy()
                    target_step_image = env.render(mode='rgb_array')
                    target_action = action.clone().cpu().numpy()
                    # print(f"Step {step_idx} Image")

            naction = action.detach().to('cpu').numpy()
            action = naction[0]

            for i in range(len(action)):
                obs, reward, done, _ = env.step(action[i])
                obs_deque.append(obs[:20])
                rewards.append(reward)
                imgs.append(env.render(mode='rgb_array'))
                step_idx += 1
                pbar.update(1)
                if step_idx > max_steps:
                    done = True
                if done:
                    print(f"Reward for {target_seed}: ", max(rewards))
                    break
finally:
    # Remove hooks
    handle.remove()
    torch.cuda.empty_cache()
    
from IPython.display import Video
vwrite('out/lowdim_f922_steer.mp4', imgs)
Video('out/lowdim_f922_steer.mp4', embed=True, width=256, height=256)

Seed 92400 Eval: 201it [00:19, 10.55it/s]                         

Reward for 92400:  0.9363688938695475


In [15]:
target_seed = int(seed)
target_step = int(step)
obs_horizon = 2
target_timestep = int(timestep)
env = PushTKeypointsEnv()
env.seed(target_seed)
np.random.seed(target_seed)
torch.manual_seed(target_seed)
obs = env.reset()
imgs = []  #
obs_deque = collections.deque([obs[:20]] * obs_horizon, maxlen=obs_horizon)
done = False
max_steps = 200
step_idx = 0
rewards = []
policy.to('cuda')

def steer_latent(layer_num, sae, steering_coefficient, latent_idx):
    def hook(module, input, output):
        output = output + steering_coefficient * sae.w_dec[latent_idx]
        return output
    return hook

layer_num = 4
handle = policy.model.decoder.layers[layer_num].register_forward_hook(steer_latent(layer_num, sae, 200, feature_idx))
try: 
    with tqdm(total=max_steps, desc=f"Seed {target_seed} Eval") as pbar:
        while not done:
            B = 1
            obs_seq = np.stack(obs_deque)
            nobs = torch.from_numpy(obs_seq).unsqueeze(0).to(device, dtype=torch.float32)

            with torch.no_grad():
                # Normalization and action inference logic from your code
                nobs = policy.normalizer['obs'].normalize(nobs)
                B, _, Do = nobs.shape
                To = policy.n_obs_steps
                T = policy.horizon
                Da = policy.action_dim
                device = policy.device
                dtype = policy.dtype
                cond = nobs[:, :To]

                shape = (B, T, Da)
                if policy.pred_action_steps_only:
                    shape = (B, policy.n_action_steps, Da)
                cond_data = torch.zeros(size=shape, device=device, dtype=dtype)
                cond_mask = torch.zeros_like(cond_data, dtype=torch.bool)

                generator = None
                trajectory = torch.randn(
                    size=cond_data.shape, 
                    dtype=cond_data.dtype,
                    device=cond_data.device,
                    generator=generator)
        
                policy.noise_scheduler.set_timesteps(policy.num_inference_steps)

                
                for t in policy.noise_scheduler.timesteps:
                    trajectory[cond_mask] = cond_data[cond_mask]

                    model_output = policy.model(trajectory, t, cond)

                    trajectory = policy.noise_scheduler.step(
                        model_output, t, trajectory, 
                        generator=generator,
                        **policy.kwargs
                    ).prev_sample

                trajectory[cond_mask] = cond_data[cond_mask]
                naction_pred = trajectory[..., :Da]
                action_pred = policy.normalizer['action'].unnormalize(naction_pred)

                start = To - 1
                end = start + policy.n_action_steps
                action = action_pred[:, start:end]

                # inference_inputs[step_idx] = []
                if step_idx == target_step:
                    target_obs = cond.clone().cpu().numpy()
                    target_step_image = env.render(mode='rgb_array')
                    target_action = action.clone().cpu().numpy()
                    # print(f"Step {step_idx} Image")

            naction = action.detach().to('cpu').numpy()
            action = naction[0]

            for i in range(len(action)):
                obs, reward, done, _ = env.step(action[i])
                obs_deque.append(obs[:20])
                rewards.append(reward)
                imgs.append(env.render(mode='rgb_array'))
                step_idx += 1
                pbar.update(1)
                if step_idx > max_steps:
                    done = True
                if done:
                    print(f"Reward for {target_seed}: ", max(rewards))
                    break
finally:
    # Remove hooks
    handle.remove()
    torch.cuda.empty_cache()
    
from IPython.display import Video
vwrite('out/lowdim_f922_steer.mp4', imgs)
Video('out/lowdim_f922_steer.mp4', embed=True, width=256, height=256)

Seed 92400 Eval: 201it [00:19, 10.46it/s]                         

Reward for 92400:  0.5067001300938854


In [16]:
target_seed = int(seed)
target_step = int(step)
obs_horizon = 2
target_timestep = int(timestep)
env = PushTKeypointsEnv()
env.seed(target_seed)
np.random.seed(target_seed)
torch.manual_seed(target_seed)
obs = env.reset()
imgs = []  #
obs_deque = collections.deque([obs[:20]] * obs_horizon, maxlen=obs_horizon)
done = False
max_steps = 200
step_idx = 0
rewards = []
policy.to('cuda')

def steer_latent(layer_num, sae, steering_coefficient, latent_idx):
    def hook(module, input, output):
        output = output + steering_coefficient * sae.w_dec[latent_idx]
        return output
    return hook

layer_num = 4
handle = policy.model.decoder.layers[layer_num].register_forward_hook(steer_latent(layer_num, sae, 30, feature_idx))
try: 
    with tqdm(total=max_steps, desc=f"Seed {target_seed} Eval") as pbar:
        while not done:
            B = 1
            obs_seq = np.stack(obs_deque)
            nobs = torch.from_numpy(obs_seq).unsqueeze(0).to(device, dtype=torch.float32)

            with torch.no_grad():
                # Normalization and action inference logic from your code
                nobs = policy.normalizer['obs'].normalize(nobs)
                B, _, Do = nobs.shape
                To = policy.n_obs_steps
                T = policy.horizon
                Da = policy.action_dim
                device = policy.device
                dtype = policy.dtype
                cond = nobs[:, :To]

                shape = (B, T, Da)
                if policy.pred_action_steps_only:
                    shape = (B, policy.n_action_steps, Da)
                cond_data = torch.zeros(size=shape, device=device, dtype=dtype)
                cond_mask = torch.zeros_like(cond_data, dtype=torch.bool)

                generator = None
                trajectory = torch.randn(
                    size=cond_data.shape, 
                    dtype=cond_data.dtype,
                    device=cond_data.device,
                    generator=generator)
        
                policy.noise_scheduler.set_timesteps(policy.num_inference_steps)

                
                for t in policy.noise_scheduler.timesteps:
                    trajectory[cond_mask] = cond_data[cond_mask]

                    model_output = policy.model(trajectory, t, cond)

                    trajectory = policy.noise_scheduler.step(
                        model_output, t, trajectory, 
                        generator=generator,
                        **policy.kwargs
                    ).prev_sample

                trajectory[cond_mask] = cond_data[cond_mask]
                naction_pred = trajectory[..., :Da]
                action_pred = policy.normalizer['action'].unnormalize(naction_pred)

                start = To - 1
                end = start + policy.n_action_steps
                action = action_pred[:, start:end]

                # inference_inputs[step_idx] = []
                if step_idx == target_step:
                    target_obs = cond.clone().cpu().numpy()
                    target_step_image = env.render(mode='rgb_array')
                    target_action = action.clone().cpu().numpy()
                    # print(f"Step {step_idx} Image")

            naction = action.detach().to('cpu').numpy()
            action = naction[0]

            for i in range(len(action)):
                obs, reward, done, _ = env.step(action[i])
                obs_deque.append(obs[:20])
                rewards.append(reward)
                imgs.append(env.render(mode='rgb_array'))
                step_idx += 1
                pbar.update(1)
                if step_idx > max_steps:
                    done = True
                if done:
                    print(f"Reward for {target_seed}: ", max(rewards))
                    break
finally:
    # Remove hooks
    handle.remove()
    torch.cuda.empty_cache()
    
from IPython.display import Video
vwrite('out/lowdim_f922_steer.mp4', imgs)
Video('out/lowdim_f922_steer.mp4', embed=True, width=256, height=256)

Seed 92400 Eval: 201it [00:19, 10.55it/s]                         

Reward for 92400:  0.8404782342375825


In [17]:
target_seed = int(seed)
target_step = int(step)
obs_horizon = 2
target_timestep = int(timestep)
env = PushTKeypointsEnv()
env.seed(target_seed)
np.random.seed(target_seed)
torch.manual_seed(target_seed)
obs = env.reset()
imgs = []  #
obs_deque = collections.deque([obs[:20]] * obs_horizon, maxlen=obs_horizon)
done = False
max_steps = 200
step_idx = 0
rewards = []
policy.to('cuda')

def steer_latent(layer_num, sae, steering_coefficient, latent_idx):
    def hook(module, input, output):
        output = output + steering_coefficient * sae.w_dec[latent_idx]
        return output
    return hook

layer_num = 4
handle = policy.model.decoder.layers[layer_num].register_forward_hook(steer_latent(layer_num, sae, 30, 1198))
try: 
    with tqdm(total=max_steps, desc=f"Seed {target_seed} Eval") as pbar:
        while not done:
            B = 1
            obs_seq = np.stack(obs_deque)
            nobs = torch.from_numpy(obs_seq).unsqueeze(0).to(device, dtype=torch.float32)

            with torch.no_grad():
                # Normalization and action inference logic from your code
                nobs = policy.normalizer['obs'].normalize(nobs)
                B, _, Do = nobs.shape
                To = policy.n_obs_steps
                T = policy.horizon
                Da = policy.action_dim
                device = policy.device
                dtype = policy.dtype
                cond = nobs[:, :To]

                shape = (B, T, Da)
                if policy.pred_action_steps_only:
                    shape = (B, policy.n_action_steps, Da)
                cond_data = torch.zeros(size=shape, device=device, dtype=dtype)
                cond_mask = torch.zeros_like(cond_data, dtype=torch.bool)

                generator = None
                trajectory = torch.randn(
                    size=cond_data.shape, 
                    dtype=cond_data.dtype,
                    device=cond_data.device,
                    generator=generator)
        
                policy.noise_scheduler.set_timesteps(policy.num_inference_steps)

                
                for t in policy.noise_scheduler.timesteps:
                    trajectory[cond_mask] = cond_data[cond_mask]

                    model_output = policy.model(trajectory, t, cond)

                    trajectory = policy.noise_scheduler.step(
                        model_output, t, trajectory, 
                        generator=generator,
                        **policy.kwargs
                    ).prev_sample

                trajectory[cond_mask] = cond_data[cond_mask]
                naction_pred = trajectory[..., :Da]
                action_pred = policy.normalizer['action'].unnormalize(naction_pred)

                start = To - 1
                end = start + policy.n_action_steps
                action = action_pred[:, start:end]

                # inference_inputs[step_idx] = []
                if step_idx == target_step:
                    target_obs = cond.clone().cpu().numpy()
                    target_step_image = env.render(mode='rgb_array')
                    target_action = action.clone().cpu().numpy()
                    # print(f"Step {step_idx} Image")

            naction = action.detach().to('cpu').numpy()
            action = naction[0]

            for i in range(len(action)):
                obs, reward, done, _ = env.step(action[i])
                obs_deque.append(obs[:20])
                rewards.append(reward)
                imgs.append(env.render(mode='rgb_array'))
                step_idx += 1
                pbar.update(1)
                if step_idx > max_steps:
                    done = True
                if done:
                    print(f"Reward for {target_seed}: ", max(rewards))
                    break
finally:
    # Remove hooks
    handle.remove()
    torch.cuda.empty_cache()
    
from IPython.display import Video
vwrite('out/lowdim_f922_steer.mp4', imgs)
Video('out/lowdim_f922_steer.mp4', embed=True, width=256, height=256)

Seed 92400 Eval: 201it [00:19, 10.47it/s]                         

Reward for 92400:  0.47449344624404627


In [18]:
target_seed = int(seed)
target_step = int(step)
obs_horizon = 2
target_timestep = int(timestep)
env = PushTKeypointsEnv()
env.seed(target_seed)
np.random.seed(target_seed)
torch.manual_seed(target_seed)
obs = env.reset()
imgs = []  #
obs_deque = collections.deque([obs[:20]] * obs_horizon, maxlen=obs_horizon)
done = False
max_steps = 200
step_idx = 0
rewards = []
policy.to('cuda')

def steer_latent(layer_num, sae, steering_coefficient, latent_idx):
    def hook(module, input, output):
        output = output + steering_coefficient * sae.w_dec[latent_idx]
        return output
    return hook

layer_num = 4
handle = policy.model.decoder.layers[layer_num].register_forward_hook(steer_latent(layer_num, sae, 3, 1198))
try: 
    with tqdm(total=max_steps, desc=f"Seed {target_seed} Eval") as pbar:
        while not done:
            B = 1
            obs_seq = np.stack(obs_deque)
            nobs = torch.from_numpy(obs_seq).unsqueeze(0).to(device, dtype=torch.float32)

            with torch.no_grad():
                # Normalization and action inference logic from your code
                nobs = policy.normalizer['obs'].normalize(nobs)
                B, _, Do = nobs.shape
                To = policy.n_obs_steps
                T = policy.horizon
                Da = policy.action_dim
                device = policy.device
                dtype = policy.dtype
                cond = nobs[:, :To]

                shape = (B, T, Da)
                if policy.pred_action_steps_only:
                    shape = (B, policy.n_action_steps, Da)
                cond_data = torch.zeros(size=shape, device=device, dtype=dtype)
                cond_mask = torch.zeros_like(cond_data, dtype=torch.bool)

                generator = None
                trajectory = torch.randn(
                    size=cond_data.shape, 
                    dtype=cond_data.dtype,
                    device=cond_data.device,
                    generator=generator)
        
                policy.noise_scheduler.set_timesteps(policy.num_inference_steps)

                
                for t in policy.noise_scheduler.timesteps:
                    trajectory[cond_mask] = cond_data[cond_mask]

                    model_output = policy.model(trajectory, t, cond)

                    trajectory = policy.noise_scheduler.step(
                        model_output, t, trajectory, 
                        generator=generator,
                        **policy.kwargs
                    ).prev_sample

                trajectory[cond_mask] = cond_data[cond_mask]
                naction_pred = trajectory[..., :Da]
                action_pred = policy.normalizer['action'].unnormalize(naction_pred)

                start = To - 1
                end = start + policy.n_action_steps
                action = action_pred[:, start:end]

                # inference_inputs[step_idx] = []
                if step_idx == target_step:
                    target_obs = cond.clone().cpu().numpy()
                    target_step_image = env.render(mode='rgb_array')
                    target_action = action.clone().cpu().numpy()
                    # print(f"Step {step_idx} Image")

            naction = action.detach().to('cpu').numpy()
            action = naction[0]

            for i in range(len(action)):
                obs, reward, done, _ = env.step(action[i])
                obs_deque.append(obs[:20])
                rewards.append(reward)
                imgs.append(env.render(mode='rgb_array'))
                step_idx += 1
                pbar.update(1)
                if step_idx > max_steps:
                    done = True
                if done:
                    print(f"Reward for {target_seed}: ", max(rewards))
                    break
finally:
    # Remove hooks
    handle.remove()
    torch.cuda.empty_cache()
    
from IPython.display import Video
vwrite('out/lowdim_f922_steer.mp4', imgs)
Video('out/lowdim_f922_steer.mp4', embed=True, width=256, height=256)

Seed 92400 Eval:  83%|████████▎ | 166/200 [00:15<00:03, 10.75it/s]

Reward for 92400:  1.0


In [19]:
target_seed = 47600 # int(seed)
target_step = int(step)
obs_horizon = 2
target_timestep = int(timestep)
env = PushTKeypointsEnv()
env.seed(target_seed)
np.random.seed(target_seed)
torch.manual_seed(target_seed)
obs = env.reset()
imgs = []  #
obs_deque = collections.deque([obs[:20]] * obs_horizon, maxlen=obs_horizon)
done = False
max_steps = 200
step_idx = 0
rewards = []
policy.to('cuda')
# inference_inputs = {}
with tqdm(total=max_steps, desc=f"Seed {target_seed} Eval") as pbar:
    while not done:
        B = 1
        obs_seq = np.stack(obs_deque)
        nobs = torch.from_numpy(obs_seq).unsqueeze(0).to(device, dtype=torch.float32)

        with torch.no_grad():
            # Normalization and action inference logic from your code
            nobs = policy.normalizer['obs'].normalize(nobs)
            B, _, Do = nobs.shape
            To = policy.n_obs_steps
            T = policy.horizon
            Da = policy.action_dim
            device = policy.device
            dtype = policy.dtype
            cond = nobs[:, :To]

            shape = (B, T, Da)
            if policy.pred_action_steps_only:
                shape = (B, policy.n_action_steps, Da)
            cond_data = torch.zeros(size=shape, device=device, dtype=dtype)
            cond_mask = torch.zeros_like(cond_data, dtype=torch.bool)

            generator = None
            trajectory = torch.randn(
                size=cond_data.shape, 
                dtype=cond_data.dtype,
                device=cond_data.device,
                generator=generator)
    
            policy.noise_scheduler.set_timesteps(policy.num_inference_steps)

            
            for t in policy.noise_scheduler.timesteps:
                trajectory[cond_mask] = cond_data[cond_mask]

                model_output = policy.model(trajectory, t, cond)

                trajectory = policy.noise_scheduler.step(
                    model_output, t, trajectory, 
                    generator=generator,
                    **policy.kwargs
                ).prev_sample

            trajectory[cond_mask] = cond_data[cond_mask]
            naction_pred = trajectory[..., :Da]
            action_pred = policy.normalizer['action'].unnormalize(naction_pred)

            start = To - 1
            end = start + policy.n_action_steps
            action = action_pred[:, start:end]

            # inference_inputs[step_idx] = []
            if step_idx == target_step:
                target_obs = cond.clone().cpu().numpy()
                target_step_image = env.render(mode='rgb_array')
                target_action = action.clone().cpu().numpy()
                # print(f"Step {step_idx} Image")

        naction = action.detach().to('cpu').numpy()
        action = naction[0]

        for i in range(len(action)):
            obs, reward, done, _ = env.step(action[i])
            obs_deque.append(obs[:20])
            rewards.append(reward)
            imgs.append(env.render(mode='rgb_array'))
            step_idx += 1
            pbar.update(1)
            if step_idx > max_steps:
                done = True
            if done:
                print(f"Reward for {target_seed}: ", max(rewards))
                break

from IPython.display import Video
vwrite(f'out/lowdim_seed{seed}_og.mp4', imgs)
Video(f'out/lowdim_seed{seed}_og.mp4', embed=True, width=256, height=256)

Seed 47600 Eval: 201it [00:19, 10.57it/s]                         

Reward for 47600:  0.008424025130009852


In [20]:
target_seed = 47600 #int(seed)
target_step = int(step)
obs_horizon = 2
target_timestep = int(timestep)
env = PushTKeypointsEnv()
env.seed(target_seed)
np.random.seed(target_seed)
torch.manual_seed(target_seed)
obs = env.reset()
imgs = []  #
obs_deque = collections.deque([obs[:20]] * obs_horizon, maxlen=obs_horizon)
done = False
max_steps = 200
step_idx = 0
rewards = []
policy.to('cuda')

def steer_latent(layer_num, sae, steering_coefficient, latent_idx):
    def hook(module, input, output):
        output = output + steering_coefficient * sae.w_dec[latent_idx]
        return output
    return hook
sfeature_idx = 1198
layer_num = 4
handle = policy.model.decoder.layers[layer_num].register_forward_hook(steer_latent(layer_num, sae, 6, sfeature_idx))
try: 
    with tqdm(total=max_steps, desc=f"Seed {target_seed} Eval") as pbar:
        while not done:
            B = 1
            obs_seq = np.stack(obs_deque)
            nobs = torch.from_numpy(obs_seq).unsqueeze(0).to(device, dtype=torch.float32)

            with torch.no_grad():
                # Normalization and action inference logic from your code
                nobs = policy.normalizer['obs'].normalize(nobs)
                B, _, Do = nobs.shape
                To = policy.n_obs_steps
                T = policy.horizon
                Da = policy.action_dim
                device = policy.device
                dtype = policy.dtype
                cond = nobs[:, :To]

                shape = (B, T, Da)
                if policy.pred_action_steps_only:
                    shape = (B, policy.n_action_steps, Da)
                cond_data = torch.zeros(size=shape, device=device, dtype=dtype)
                cond_mask = torch.zeros_like(cond_data, dtype=torch.bool)

                generator = None
                trajectory = torch.randn(
                    size=cond_data.shape, 
                    dtype=cond_data.dtype,
                    device=cond_data.device,
                    generator=generator)
        
                policy.noise_scheduler.set_timesteps(policy.num_inference_steps)

                
                for t in policy.noise_scheduler.timesteps:
                    trajectory[cond_mask] = cond_data[cond_mask]

                    model_output = policy.model(trajectory, t, cond)

                    trajectory = policy.noise_scheduler.step(
                        model_output, t, trajectory, 
                        generator=generator,
                        **policy.kwargs
                    ).prev_sample

                trajectory[cond_mask] = cond_data[cond_mask]
                naction_pred = trajectory[..., :Da]
                action_pred = policy.normalizer['action'].unnormalize(naction_pred)

                start = To - 1
                end = start + policy.n_action_steps
                action = action_pred[:, start:end]

                # inference_inputs[step_idx] = []
                if step_idx == target_step:
                    target_obs = cond.clone().cpu().numpy()
                    target_step_image = env.render(mode='rgb_array')
                    target_action = action.clone().cpu().numpy()
                    # print(f"Step {step_idx} Image")

            naction = action.detach().to('cpu').numpy()
            action = naction[0]

            for i in range(len(action)):
                obs, reward, done, _ = env.step(action[i])
                obs_deque.append(obs[:20])
                rewards.append(reward)
                imgs.append(env.render(mode='rgb_array'))
                step_idx += 1
                pbar.update(1)
                if step_idx > max_steps:
                    done = True
                if done:
                    print(f"Reward for {target_seed}: ", max(rewards))
                    break
finally:
    # Remove hooks
    handle.remove()
    torch.cuda.empty_cache()
    
from IPython.display import Video
vwrite(f'out/lowdim_seed{seed}_f{sfeature_idx}_steer.mp4', imgs)
Video(f'out/lowdim_seed{seed}_f{sfeature_idx}_steer.mp4', embed=True, width=256, height=256)

Seed 47600 Eval: 201it [00:19, 10.58it/s]                         

Reward for 47600:  0.4857360357084123


In [21]:
target_seed = 37500 # int(seed)
target_step = int(step)
obs_horizon = 2
target_timestep = int(timestep)
env = PushTKeypointsEnv()
env.seed(target_seed)
np.random.seed(target_seed)
torch.manual_seed(target_seed)
obs = env.reset()
imgs = []  #
obs_deque = collections.deque([obs[:20]] * obs_horizon, maxlen=obs_horizon)
done = False
max_steps = 200
step_idx = 0
rewards = []
policy.to('cuda')
# inference_inputs = {}
with tqdm(total=max_steps, desc=f"Seed {target_seed} Eval") as pbar:
    while not done:
        B = 1
        obs_seq = np.stack(obs_deque)
        nobs = torch.from_numpy(obs_seq).unsqueeze(0).to(device, dtype=torch.float32)

        with torch.no_grad():
            # Normalization and action inference logic from your code
            nobs = policy.normalizer['obs'].normalize(nobs)
            B, _, Do = nobs.shape
            To = policy.n_obs_steps
            T = policy.horizon
            Da = policy.action_dim
            device = policy.device
            dtype = policy.dtype
            cond = nobs[:, :To]

            shape = (B, T, Da)
            if policy.pred_action_steps_only:
                shape = (B, policy.n_action_steps, Da)
            cond_data = torch.zeros(size=shape, device=device, dtype=dtype)
            cond_mask = torch.zeros_like(cond_data, dtype=torch.bool)

            generator = None
            trajectory = torch.randn(
                size=cond_data.shape, 
                dtype=cond_data.dtype,
                device=cond_data.device,
                generator=generator)
    
            policy.noise_scheduler.set_timesteps(policy.num_inference_steps)

            
            for t in policy.noise_scheduler.timesteps:
                trajectory[cond_mask] = cond_data[cond_mask]

                model_output = policy.model(trajectory, t, cond)

                trajectory = policy.noise_scheduler.step(
                    model_output, t, trajectory, 
                    generator=generator,
                    **policy.kwargs
                ).prev_sample

            trajectory[cond_mask] = cond_data[cond_mask]
            naction_pred = trajectory[..., :Da]
            action_pred = policy.normalizer['action'].unnormalize(naction_pred)

            start = To - 1
            end = start + policy.n_action_steps
            action = action_pred[:, start:end]

            # inference_inputs[step_idx] = []
            if step_idx == target_step:
                target_obs = cond.clone().cpu().numpy()
                target_step_image = env.render(mode='rgb_array')
                target_action = action.clone().cpu().numpy()
                # print(f"Step {step_idx} Image")

        naction = action.detach().to('cpu').numpy()
        action = naction[0]

        for i in range(len(action)):
            obs, reward, done, _ = env.step(action[i])
            obs_deque.append(obs[:20])
            rewards.append(reward)
            imgs.append(env.render(mode='rgb_array'))
            step_idx += 1
            pbar.update(1)
            if step_idx > max_steps:
                done = True
            if done:
                print(f"Reward for {target_seed}: ", max(rewards))
                break

from IPython.display import Video
vwrite(f'out/lowdim_seed{target_seed}_og.mp4', imgs)
Video(f'out/lowdim_seed{target_seed}_og.mp4', embed=True, width=256, height=256)

Seed 37500 Eval: 201it [00:18, 10.63it/s]                         

Reward for 37500:  0.9090183553707739


In [22]:
target_seed = 37500 #int(seed)
target_step = int(step)
obs_horizon = 2
target_timestep = int(timestep)
env = PushTKeypointsEnv()
env.seed(target_seed)
np.random.seed(target_seed)
torch.manual_seed(target_seed)
obs = env.reset()
imgs = []  #
obs_deque = collections.deque([obs[:20]] * obs_horizon, maxlen=obs_horizon)
done = False
max_steps = 200
step_idx = 0
rewards = []
policy.to('cuda')

def steer_latent(layer_num, sae, steering_coefficient, latent_idx):
    def hook(module, input, output):
        output = output + steering_coefficient * sae.w_dec[latent_idx]
        return output
    return hook
sfeature_idx = 1198
layer_num = 4
handle = policy.model.decoder.layers[layer_num].register_forward_hook(steer_latent(layer_num, sae, 6, sfeature_idx))
try: 
    with tqdm(total=max_steps, desc=f"Seed {target_seed} Eval") as pbar:
        while not done:
            B = 1
            obs_seq = np.stack(obs_deque)
            nobs = torch.from_numpy(obs_seq).unsqueeze(0).to(device, dtype=torch.float32)

            with torch.no_grad():
                # Normalization and action inference logic from your code
                nobs = policy.normalizer['obs'].normalize(nobs)
                B, _, Do = nobs.shape
                To = policy.n_obs_steps
                T = policy.horizon
                Da = policy.action_dim
                device = policy.device
                dtype = policy.dtype
                cond = nobs[:, :To]

                shape = (B, T, Da)
                if policy.pred_action_steps_only:
                    shape = (B, policy.n_action_steps, Da)
                cond_data = torch.zeros(size=shape, device=device, dtype=dtype)
                cond_mask = torch.zeros_like(cond_data, dtype=torch.bool)

                generator = None
                trajectory = torch.randn(
                    size=cond_data.shape, 
                    dtype=cond_data.dtype,
                    device=cond_data.device,
                    generator=generator)
        
                policy.noise_scheduler.set_timesteps(policy.num_inference_steps)

                
                for t in policy.noise_scheduler.timesteps:
                    trajectory[cond_mask] = cond_data[cond_mask]

                    model_output = policy.model(trajectory, t, cond)

                    trajectory = policy.noise_scheduler.step(
                        model_output, t, trajectory, 
                        generator=generator,
                        **policy.kwargs
                    ).prev_sample

                trajectory[cond_mask] = cond_data[cond_mask]
                naction_pred = trajectory[..., :Da]
                action_pred = policy.normalizer['action'].unnormalize(naction_pred)

                start = To - 1
                end = start + policy.n_action_steps
                action = action_pred[:, start:end]

                # inference_inputs[step_idx] = []
                if step_idx == target_step:
                    target_obs = cond.clone().cpu().numpy()
                    target_step_image = env.render(mode='rgb_array')
                    target_action = action.clone().cpu().numpy()
                    # print(f"Step {step_idx} Image")

            naction = action.detach().to('cpu').numpy()
            action = naction[0]

            for i in range(len(action)):
                obs, reward, done, _ = env.step(action[i])
                obs_deque.append(obs[:20])
                rewards.append(reward)
                imgs.append(env.render(mode='rgb_array'))
                step_idx += 1
                pbar.update(1)
                if step_idx > max_steps:
                    done = True
                if done:
                    print(f"Reward for {target_seed}: ", max(rewards))
                    break
finally:
    # Remove hooks
    handle.remove()
    torch.cuda.empty_cache()
    
from IPython.display import Video
vwrite(f'out/lowdim_seed{target_seed}_f{sfeature_idx}_steer.mp4', imgs)
Video(f'out/lowdim_seed{target_seed}_f{sfeature_idx}_steer.mp4', embed=True, width=256, height=256)

Seed 37500 Eval: 201it [00:19, 10.55it/s]                         

Reward for 37500:  0.9258154578895814


In [23]:
target_seed = 40000 # int(seed)
target_step = int(step)
obs_horizon = 2
target_timestep = int(timestep)
env = PushTKeypointsEnv()
env.seed(target_seed)
np.random.seed(target_seed)
torch.manual_seed(target_seed)
obs = env.reset()
imgs = []  #
obs_deque = collections.deque([obs[:20]] * obs_horizon, maxlen=obs_horizon)
done = False
max_steps = 200
step_idx = 0
rewards = []
policy.to('cuda')
# inference_inputs = {}
with tqdm(total=max_steps, desc=f"Seed {target_seed} Eval") as pbar:
    while not done:
        B = 1
        obs_seq = np.stack(obs_deque)
        nobs = torch.from_numpy(obs_seq).unsqueeze(0).to(device, dtype=torch.float32)

        with torch.no_grad():
            # Normalization and action inference logic from your code
            nobs = policy.normalizer['obs'].normalize(nobs)
            B, _, Do = nobs.shape
            To = policy.n_obs_steps
            T = policy.horizon
            Da = policy.action_dim
            device = policy.device
            dtype = policy.dtype
            cond = nobs[:, :To]

            shape = (B, T, Da)
            if policy.pred_action_steps_only:
                shape = (B, policy.n_action_steps, Da)
            cond_data = torch.zeros(size=shape, device=device, dtype=dtype)
            cond_mask = torch.zeros_like(cond_data, dtype=torch.bool)

            generator = None
            trajectory = torch.randn(
                size=cond_data.shape, 
                dtype=cond_data.dtype,
                device=cond_data.device,
                generator=generator)
    
            policy.noise_scheduler.set_timesteps(policy.num_inference_steps)

            
            for t in policy.noise_scheduler.timesteps:
                trajectory[cond_mask] = cond_data[cond_mask]

                model_output = policy.model(trajectory, t, cond)

                trajectory = policy.noise_scheduler.step(
                    model_output, t, trajectory, 
                    generator=generator,
                    **policy.kwargs
                ).prev_sample

            trajectory[cond_mask] = cond_data[cond_mask]
            naction_pred = trajectory[..., :Da]
            action_pred = policy.normalizer['action'].unnormalize(naction_pred)

            start = To - 1
            end = start + policy.n_action_steps
            action = action_pred[:, start:end]

            # inference_inputs[step_idx] = []
            if step_idx == target_step:
                target_obs = cond.clone().cpu().numpy()
                target_step_image = env.render(mode='rgb_array')
                target_action = action.clone().cpu().numpy()
                # print(f"Step {step_idx} Image")

        naction = action.detach().to('cpu').numpy()
        action = naction[0]

        for i in range(len(action)):
            obs, reward, done, _ = env.step(action[i])
            obs_deque.append(obs[:20])
            rewards.append(reward)
            imgs.append(env.render(mode='rgb_array'))
            step_idx += 1
            pbar.update(1)
            if step_idx > max_steps:
                done = True
            if done:
                print(f"Reward for {target_seed}: ", max(rewards))
                break

from IPython.display import Video
vwrite(f'out/lowdim_seed{target_seed}_og.mp4', imgs)
Video(f'out/lowdim_seed{target_seed}_og.mp4', embed=True, width=256, height=256)

Seed 40000 Eval:  72%|███████▏  | 143/200 [00:13<00:05, 10.80it/s]

Reward for 40000:  1.0


In [24]:
# target_seed = 37500 #int(seed)
target_step = int(step)
obs_horizon = 2
target_timestep = int(timestep)
env = PushTKeypointsEnv()
env.seed(target_seed)
np.random.seed(target_seed)
torch.manual_seed(target_seed)
obs = env.reset()
imgs = []  #
obs_deque = collections.deque([obs[:20]] * obs_horizon, maxlen=obs_horizon)
done = False
max_steps = 200
step_idx = 0
rewards = []
policy.to('cuda')

def steer_latent(layer_num, sae, steering_coefficient, latent_idx):
    def hook(module, input, output):
        output = output + steering_coefficient * sae.w_dec[latent_idx]
        return output
    return hook
sfeature_idx = 1198
layer_num = 4
handle = policy.model.decoder.layers[layer_num].register_forward_hook(steer_latent(layer_num, sae, 6, sfeature_idx))
try: 
    with tqdm(total=max_steps, desc=f"Seed {target_seed} Eval") as pbar:
        while not done:
            B = 1
            obs_seq = np.stack(obs_deque)
            nobs = torch.from_numpy(obs_seq).unsqueeze(0).to(device, dtype=torch.float32)

            with torch.no_grad():
                # Normalization and action inference logic from your code
                nobs = policy.normalizer['obs'].normalize(nobs)
                B, _, Do = nobs.shape
                To = policy.n_obs_steps
                T = policy.horizon
                Da = policy.action_dim
                device = policy.device
                dtype = policy.dtype
                cond = nobs[:, :To]

                shape = (B, T, Da)
                if policy.pred_action_steps_only:
                    shape = (B, policy.n_action_steps, Da)
                cond_data = torch.zeros(size=shape, device=device, dtype=dtype)
                cond_mask = torch.zeros_like(cond_data, dtype=torch.bool)

                generator = None
                trajectory = torch.randn(
                    size=cond_data.shape, 
                    dtype=cond_data.dtype,
                    device=cond_data.device,
                    generator=generator)
        
                policy.noise_scheduler.set_timesteps(policy.num_inference_steps)

                
                for t in policy.noise_scheduler.timesteps:
                    trajectory[cond_mask] = cond_data[cond_mask]

                    model_output = policy.model(trajectory, t, cond)

                    trajectory = policy.noise_scheduler.step(
                        model_output, t, trajectory, 
                        generator=generator,
                        **policy.kwargs
                    ).prev_sample

                trajectory[cond_mask] = cond_data[cond_mask]
                naction_pred = trajectory[..., :Da]
                action_pred = policy.normalizer['action'].unnormalize(naction_pred)

                start = To - 1
                end = start + policy.n_action_steps
                action = action_pred[:, start:end]

                # inference_inputs[step_idx] = []
                if step_idx == target_step:
                    target_obs = cond.clone().cpu().numpy()
                    target_step_image = env.render(mode='rgb_array')
                    target_action = action.clone().cpu().numpy()
                    # print(f"Step {step_idx} Image")

            naction = action.detach().to('cpu').numpy()
            action = naction[0]

            for i in range(len(action)):
                obs, reward, done, _ = env.step(action[i])
                obs_deque.append(obs[:20])
                rewards.append(reward)
                imgs.append(env.render(mode='rgb_array'))
                step_idx += 1
                pbar.update(1)
                if step_idx > max_steps:
                    done = True
                if done:
                    print(f"Reward for {target_seed}: ", max(rewards))
                    break
finally:
    # Remove hooks
    handle.remove()
    torch.cuda.empty_cache()
    
from IPython.display import Video
vwrite(f'out/lowdim_seed{target_seed}_f{sfeature_idx}_steer.mp4', imgs)
Video(f'out/lowdim_seed{target_seed}_f{sfeature_idx}_steer.mp4', embed=True, width=256, height=256)

Seed 40000 Eval:  60%|█████▉    | 119/200 [00:11<00:07, 10.49it/s]

Reward for 40000:  1.0


In [25]:
target_seed = 52800 # int(seed)
target_step = int(step)
obs_horizon = 2
target_timestep = int(timestep)
env = PushTKeypointsEnv()
env.seed(target_seed)
np.random.seed(target_seed)
torch.manual_seed(target_seed)
obs = env.reset()
imgs = []  #
obs_deque = collections.deque([obs[:20]] * obs_horizon, maxlen=obs_horizon)
done = False
max_steps = 200
step_idx = 0
rewards = []
policy.to('cuda')
# inference_inputs = {}
with tqdm(total=max_steps, desc=f"Seed {target_seed} Eval") as pbar:
    while not done:
        B = 1
        obs_seq = np.stack(obs_deque)
        nobs = torch.from_numpy(obs_seq).unsqueeze(0).to(device, dtype=torch.float32)

        with torch.no_grad():
            # Normalization and action inference logic from your code
            nobs = policy.normalizer['obs'].normalize(nobs)
            B, _, Do = nobs.shape
            To = policy.n_obs_steps
            T = policy.horizon
            Da = policy.action_dim
            device = policy.device
            dtype = policy.dtype
            cond = nobs[:, :To]

            shape = (B, T, Da)
            if policy.pred_action_steps_only:
                shape = (B, policy.n_action_steps, Da)
            cond_data = torch.zeros(size=shape, device=device, dtype=dtype)
            cond_mask = torch.zeros_like(cond_data, dtype=torch.bool)

            generator = None
            trajectory = torch.randn(
                size=cond_data.shape, 
                dtype=cond_data.dtype,
                device=cond_data.device,
                generator=generator)
    
            policy.noise_scheduler.set_timesteps(policy.num_inference_steps)

            
            for t in policy.noise_scheduler.timesteps:
                trajectory[cond_mask] = cond_data[cond_mask]

                model_output = policy.model(trajectory, t, cond)

                trajectory = policy.noise_scheduler.step(
                    model_output, t, trajectory, 
                    generator=generator,
                    **policy.kwargs
                ).prev_sample

            trajectory[cond_mask] = cond_data[cond_mask]
            naction_pred = trajectory[..., :Da]
            action_pred = policy.normalizer['action'].unnormalize(naction_pred)

            start = To - 1
            end = start + policy.n_action_steps
            action = action_pred[:, start:end]

            # inference_inputs[step_idx] = []
            if step_idx == target_step:
                target_obs = cond.clone().cpu().numpy()
                target_step_image = env.render(mode='rgb_array')
                target_action = action.clone().cpu().numpy()
                # print(f"Step {step_idx} Image")

        naction = action.detach().to('cpu').numpy()
        action = naction[0]

        for i in range(len(action)):
            obs, reward, done, _ = env.step(action[i])
            obs_deque.append(obs[:20])
            rewards.append(reward)
            imgs.append(env.render(mode='rgb_array'))
            step_idx += 1
            pbar.update(1)
            if step_idx > max_steps:
                done = True
            if done:
                print(f"Reward for {target_seed}: ", max(rewards))
                break

from IPython.display import Video
vwrite(f'out/lowdim_seed{target_seed}_og.mp4', imgs)
Video(f'out/lowdim_seed{target_seed}_og.mp4', embed=True, width=256, height=256)

Seed 52800 Eval:  55%|█████▍    | 109/200 [00:10<00:08, 10.32it/s]

Reward for 52800:  1.0


In [26]:
# target_seed = 37500 #int(seed)
target_step = int(step)
obs_horizon = 2
target_timestep = int(timestep)
env = PushTKeypointsEnv()
env.seed(target_seed)
np.random.seed(target_seed)
torch.manual_seed(target_seed)
obs = env.reset()
imgs = []  #
obs_deque = collections.deque([obs[:20]] * obs_horizon, maxlen=obs_horizon)
done = False
max_steps = 200
step_idx = 0
rewards = []
policy.to('cuda')

def steer_latent(layer_num, sae, steering_coefficient, latent_idx):
    def hook(module, input, output):
        output = output + steering_coefficient * sae.w_dec[latent_idx]
        return output
    return hook
sfeature_idx = 1198
layer_num = 4
handle = policy.model.decoder.layers[layer_num].register_forward_hook(steer_latent(layer_num, sae, 6, sfeature_idx))
try: 
    with tqdm(total=max_steps, desc=f"Seed {target_seed} Eval") as pbar:
        while not done:
            B = 1
            obs_seq = np.stack(obs_deque)
            nobs = torch.from_numpy(obs_seq).unsqueeze(0).to(device, dtype=torch.float32)

            with torch.no_grad():
                # Normalization and action inference logic from your code
                nobs = policy.normalizer['obs'].normalize(nobs)
                B, _, Do = nobs.shape
                To = policy.n_obs_steps
                T = policy.horizon
                Da = policy.action_dim
                device = policy.device
                dtype = policy.dtype
                cond = nobs[:, :To]

                shape = (B, T, Da)
                if policy.pred_action_steps_only:
                    shape = (B, policy.n_action_steps, Da)
                cond_data = torch.zeros(size=shape, device=device, dtype=dtype)
                cond_mask = torch.zeros_like(cond_data, dtype=torch.bool)

                generator = None
                trajectory = torch.randn(
                    size=cond_data.shape, 
                    dtype=cond_data.dtype,
                    device=cond_data.device,
                    generator=generator)
        
                policy.noise_scheduler.set_timesteps(policy.num_inference_steps)

                
                for t in policy.noise_scheduler.timesteps:
                    trajectory[cond_mask] = cond_data[cond_mask]

                    model_output = policy.model(trajectory, t, cond)

                    trajectory = policy.noise_scheduler.step(
                        model_output, t, trajectory, 
                        generator=generator,
                        **policy.kwargs
                    ).prev_sample

                trajectory[cond_mask] = cond_data[cond_mask]
                naction_pred = trajectory[..., :Da]
                action_pred = policy.normalizer['action'].unnormalize(naction_pred)

                start = To - 1
                end = start + policy.n_action_steps
                action = action_pred[:, start:end]

                # inference_inputs[step_idx] = []
                if step_idx == target_step:
                    target_obs = cond.clone().cpu().numpy()
                    target_step_image = env.render(mode='rgb_array')
                    target_action = action.clone().cpu().numpy()
                    # print(f"Step {step_idx} Image")

            naction = action.detach().to('cpu').numpy()
            action = naction[0]

            for i in range(len(action)):
                obs, reward, done, _ = env.step(action[i])
                obs_deque.append(obs[:20])
                rewards.append(reward)
                imgs.append(env.render(mode='rgb_array'))
                step_idx += 1
                pbar.update(1)
                if step_idx > max_steps:
                    done = True
                if done:
                    print(f"Reward for {target_seed}: ", max(rewards))
                    break
finally:
    # Remove hooks
    handle.remove()
    torch.cuda.empty_cache()
    
from IPython.display import Video
vwrite(f'out/lowdim_seed{target_seed}_f{sfeature_idx}_steer.mp4', imgs)
Video(f'out/lowdim_seed{target_seed}_f{sfeature_idx}_steer.mp4', embed=True, width=256, height=256)

Seed 52800 Eval:  76%|███████▌  | 152/200 [00:14<00:04, 10.66it/s]

Reward for 52800:  1.0


In [27]:
target_seed = 10300 # int(seed)
target_step = int(step)
obs_horizon = 2
target_timestep = int(timestep)
env = PushTKeypointsEnv()
env.seed(target_seed)
np.random.seed(target_seed)
torch.manual_seed(target_seed)
obs = env.reset()
imgs = []  #
obs_deque = collections.deque([obs[:20]] * obs_horizon, maxlen=obs_horizon)
done = False
max_steps = 200
step_idx = 0
rewards = []
policy.to('cuda')
# inference_inputs = {}
with tqdm(total=max_steps, desc=f"Seed {target_seed} Eval") as pbar:
    while not done:
        B = 1
        obs_seq = np.stack(obs_deque)
        nobs = torch.from_numpy(obs_seq).unsqueeze(0).to(device, dtype=torch.float32)

        with torch.no_grad():
            # Normalization and action inference logic from your code
            nobs = policy.normalizer['obs'].normalize(nobs)
            B, _, Do = nobs.shape
            To = policy.n_obs_steps
            T = policy.horizon
            Da = policy.action_dim
            device = policy.device
            dtype = policy.dtype
            cond = nobs[:, :To]

            shape = (B, T, Da)
            if policy.pred_action_steps_only:
                shape = (B, policy.n_action_steps, Da)
            cond_data = torch.zeros(size=shape, device=device, dtype=dtype)
            cond_mask = torch.zeros_like(cond_data, dtype=torch.bool)

            generator = None
            trajectory = torch.randn(
                size=cond_data.shape, 
                dtype=cond_data.dtype,
                device=cond_data.device,
                generator=generator)
    
            policy.noise_scheduler.set_timesteps(policy.num_inference_steps)

            
            for t in policy.noise_scheduler.timesteps:
                trajectory[cond_mask] = cond_data[cond_mask]

                model_output = policy.model(trajectory, t, cond)

                trajectory = policy.noise_scheduler.step(
                    model_output, t, trajectory, 
                    generator=generator,
                    **policy.kwargs
                ).prev_sample

            trajectory[cond_mask] = cond_data[cond_mask]
            naction_pred = trajectory[..., :Da]
            action_pred = policy.normalizer['action'].unnormalize(naction_pred)

            start = To - 1
            end = start + policy.n_action_steps
            action = action_pred[:, start:end]

            # inference_inputs[step_idx] = []
            if step_idx == target_step:
                target_obs = cond.clone().cpu().numpy()
                target_step_image = env.render(mode='rgb_array')
                target_action = action.clone().cpu().numpy()
                # print(f"Step {step_idx} Image")

        naction = action.detach().to('cpu').numpy()
        action = naction[0]

        for i in range(len(action)):
            obs, reward, done, _ = env.step(action[i])
            obs_deque.append(obs[:20])
            rewards.append(reward)
            imgs.append(env.render(mode='rgb_array'))
            step_idx += 1
            pbar.update(1)
            if step_idx > max_steps:
                done = True
            if done:
                print(f"Reward for {target_seed}: ", max(rewards))
                break

from IPython.display import Video
vwrite(f'out/lowdim_seed{target_seed}_og.mp4', imgs)
Video(f'out/lowdim_seed{target_seed}_og.mp4', embed=True, width=256, height=256)

Seed 10300 Eval:  98%|█████████▊| 195/200 [00:18<00:00, 10.36it/s]

Reward for 10300:  1.0


In [28]:
# target_seed = 37500 #int(seed)
target_step = int(step)
obs_horizon = 2
target_timestep = int(timestep)
env = PushTKeypointsEnv()
env.seed(target_seed)
np.random.seed(target_seed)
torch.manual_seed(target_seed)
obs = env.reset()
imgs = []  #
obs_deque = collections.deque([obs[:20]] * obs_horizon, maxlen=obs_horizon)
done = False
max_steps = 200
step_idx = 0
rewards = []
policy.to('cuda')

def steer_latent(layer_num, sae, steering_coefficient, latent_idx):
    def hook(module, input, output):
        output = output + steering_coefficient * sae.w_dec[latent_idx]
        return output
    return hook
sfeature_idx = 1198
layer_num = 4
handle = policy.model.decoder.layers[layer_num].register_forward_hook(steer_latent(layer_num, sae, 6, sfeature_idx))
try: 
    with tqdm(total=max_steps, desc=f"Seed {target_seed} Eval") as pbar:
        while not done:
            B = 1
            obs_seq = np.stack(obs_deque)
            nobs = torch.from_numpy(obs_seq).unsqueeze(0).to(device, dtype=torch.float32)

            with torch.no_grad():
                # Normalization and action inference logic from your code
                nobs = policy.normalizer['obs'].normalize(nobs)
                B, _, Do = nobs.shape
                To = policy.n_obs_steps
                T = policy.horizon
                Da = policy.action_dim
                device = policy.device
                dtype = policy.dtype
                cond = nobs[:, :To]

                shape = (B, T, Da)
                if policy.pred_action_steps_only:
                    shape = (B, policy.n_action_steps, Da)
                cond_data = torch.zeros(size=shape, device=device, dtype=dtype)
                cond_mask = torch.zeros_like(cond_data, dtype=torch.bool)

                generator = None
                trajectory = torch.randn(
                    size=cond_data.shape, 
                    dtype=cond_data.dtype,
                    device=cond_data.device,
                    generator=generator)
        
                policy.noise_scheduler.set_timesteps(policy.num_inference_steps)

                
                for t in policy.noise_scheduler.timesteps:
                    trajectory[cond_mask] = cond_data[cond_mask]

                    model_output = policy.model(trajectory, t, cond)

                    trajectory = policy.noise_scheduler.step(
                        model_output, t, trajectory, 
                        generator=generator,
                        **policy.kwargs
                    ).prev_sample

                trajectory[cond_mask] = cond_data[cond_mask]
                naction_pred = trajectory[..., :Da]
                action_pred = policy.normalizer['action'].unnormalize(naction_pred)

                start = To - 1
                end = start + policy.n_action_steps
                action = action_pred[:, start:end]

                # inference_inputs[step_idx] = []
                if step_idx == target_step:
                    target_obs = cond.clone().cpu().numpy()
                    target_step_image = env.render(mode='rgb_array')
                    target_action = action.clone().cpu().numpy()
                    # print(f"Step {step_idx} Image")

            naction = action.detach().to('cpu').numpy()
            action = naction[0]

            for i in range(len(action)):
                obs, reward, done, _ = env.step(action[i])
                obs_deque.append(obs[:20])
                rewards.append(reward)
                imgs.append(env.render(mode='rgb_array'))
                step_idx += 1
                pbar.update(1)
                if step_idx > max_steps:
                    done = True
                if done:
                    print(f"Reward for {target_seed}: ", max(rewards))
                    break
finally:
    # Remove hooks
    handle.remove()
    torch.cuda.empty_cache()
    
from IPython.display import Video
vwrite(f'out/lowdim_seed{target_seed}_f{sfeature_idx}_steer.mp4', imgs)
Video(f'out/lowdim_seed{target_seed}_f{sfeature_idx}_steer.mp4', embed=True, width=256, height=256)

Seed 10300 Eval: 201it [00:19, 10.26it/s]                         

Reward for 10300:  0.978239118414967


In [29]:
target_seed = 92400 # int(seed)
target_step = int(step)
obs_horizon = 2
target_timestep = int(timestep)
env = PushTKeypointsEnv()
env.seed(target_seed)
np.random.seed(target_seed)
torch.manual_seed(target_seed)
obs = env.reset()
imgs = []  #
obs_deque = collections.deque([obs[:20]] * obs_horizon, maxlen=obs_horizon)
done = False
max_steps = 200
step_idx = 0
rewards = []
policy.to('cuda')
# inference_inputs = {}
with tqdm(total=max_steps, desc=f"Seed {target_seed} Eval") as pbar:
    while not done:
        B = 1
        obs_seq = np.stack(obs_deque)
        nobs = torch.from_numpy(obs_seq).unsqueeze(0).to(device, dtype=torch.float32)

        with torch.no_grad():
            # Normalization and action inference logic from your code
            nobs = policy.normalizer['obs'].normalize(nobs)
            B, _, Do = nobs.shape
            To = policy.n_obs_steps
            T = policy.horizon
            Da = policy.action_dim
            device = policy.device
            dtype = policy.dtype
            cond = nobs[:, :To]

            shape = (B, T, Da)
            if policy.pred_action_steps_only:
                shape = (B, policy.n_action_steps, Da)
            cond_data = torch.zeros(size=shape, device=device, dtype=dtype)
            cond_mask = torch.zeros_like(cond_data, dtype=torch.bool)

            generator = None
            trajectory = torch.randn(
                size=cond_data.shape, 
                dtype=cond_data.dtype,
                device=cond_data.device,
                generator=generator)
    
            policy.noise_scheduler.set_timesteps(policy.num_inference_steps)

            
            for t in policy.noise_scheduler.timesteps:
                trajectory[cond_mask] = cond_data[cond_mask]

                model_output = policy.model(trajectory, t, cond)

                trajectory = policy.noise_scheduler.step(
                    model_output, t, trajectory, 
                    generator=generator,
                    **policy.kwargs
                ).prev_sample

            trajectory[cond_mask] = cond_data[cond_mask]
            naction_pred = trajectory[..., :Da]
            action_pred = policy.normalizer['action'].unnormalize(naction_pred)

            start = To - 1
            end = start + policy.n_action_steps
            action = action_pred[:, start:end]

            # inference_inputs[step_idx] = []
            if step_idx == target_step:
                target_obs = cond.clone().cpu().numpy()
                target_step_image = env.render(mode='rgb_array')
                target_action = action.clone().cpu().numpy()
                # print(f"Step {step_idx} Image")

        naction = action.detach().to('cpu').numpy()
        action = naction[0]

        for i in range(len(action)):
            obs, reward, done, _ = env.step(action[i])
            obs_deque.append(obs[:20])
            rewards.append(reward)
            imgs.append(env.render(mode='rgb_array'))
            step_idx += 1
            pbar.update(1)
            if step_idx > max_steps:
                done = True
            if done:
                print(f"Reward for {target_seed}: ", max(rewards))
                break

from IPython.display import Video
vwrite(f'out/lowdim_seed{target_seed}_og.mp4', imgs)
Video(f'out/lowdim_seed{target_seed}_og.mp4', embed=True, width=256, height=256)

Seed 92400 Eval: 201it [00:19, 10.33it/s]                         

Reward for 92400:  0.9641604420488913


In [30]:
# target_seed = 37500 #int(seed)
target_step = int(step)
obs_horizon = 2
target_timestep = int(timestep)
env = PushTKeypointsEnv()
env.seed(target_seed)
np.random.seed(target_seed)
torch.manual_seed(target_seed)
obs = env.reset()
imgs = []  #
obs_deque = collections.deque([obs[:20]] * obs_horizon, maxlen=obs_horizon)
done = False
max_steps = 200
step_idx = 0
rewards = []
policy.to('cuda')

def steer_latent(layer_num, sae, steering_coefficient, latent_idx):
    def hook(module, input, output):
        output = output + steering_coefficient * sae.w_dec[latent_idx]
        return output
    return hook
sfeature_idx = 1198
layer_num = 4
handle = policy.model.decoder.layers[layer_num].register_forward_hook(steer_latent(layer_num, sae, 6, sfeature_idx))
try: 
    with tqdm(total=max_steps, desc=f"Seed {target_seed} Eval") as pbar:
        while not done:
            B = 1
            obs_seq = np.stack(obs_deque)
            nobs = torch.from_numpy(obs_seq).unsqueeze(0).to(device, dtype=torch.float32)

            with torch.no_grad():
                # Normalization and action inference logic from your code
                nobs = policy.normalizer['obs'].normalize(nobs)
                B, _, Do = nobs.shape
                To = policy.n_obs_steps
                T = policy.horizon
                Da = policy.action_dim
                device = policy.device
                dtype = policy.dtype
                cond = nobs[:, :To]

                shape = (B, T, Da)
                if policy.pred_action_steps_only:
                    shape = (B, policy.n_action_steps, Da)
                cond_data = torch.zeros(size=shape, device=device, dtype=dtype)
                cond_mask = torch.zeros_like(cond_data, dtype=torch.bool)

                generator = None
                trajectory = torch.randn(
                    size=cond_data.shape, 
                    dtype=cond_data.dtype,
                    device=cond_data.device,
                    generator=generator)
        
                policy.noise_scheduler.set_timesteps(policy.num_inference_steps)

                
                for t in policy.noise_scheduler.timesteps:
                    trajectory[cond_mask] = cond_data[cond_mask]

                    model_output = policy.model(trajectory, t, cond)

                    trajectory = policy.noise_scheduler.step(
                        model_output, t, trajectory, 
                        generator=generator,
                        **policy.kwargs
                    ).prev_sample

                trajectory[cond_mask] = cond_data[cond_mask]
                naction_pred = trajectory[..., :Da]
                action_pred = policy.normalizer['action'].unnormalize(naction_pred)

                start = To - 1
                end = start + policy.n_action_steps
                action = action_pred[:, start:end]

                # inference_inputs[step_idx] = []
                if step_idx == target_step:
                    target_obs = cond.clone().cpu().numpy()
                    target_step_image = env.render(mode='rgb_array')
                    target_action = action.clone().cpu().numpy()
                    # print(f"Step {step_idx} Image")

            naction = action.detach().to('cpu').numpy()
            action = naction[0]

            for i in range(len(action)):
                obs, reward, done, _ = env.step(action[i])
                obs_deque.append(obs[:20])
                rewards.append(reward)
                imgs.append(env.render(mode='rgb_array'))
                step_idx += 1
                pbar.update(1)
                if step_idx > max_steps:
                    done = True
                if done:
                    print(f"Reward for {target_seed}: ", max(rewards))
                    break
finally:
    # Remove hooks
    handle.remove()
    torch.cuda.empty_cache()
    
from IPython.display import Video
vwrite(f'out/lowdim_seed{target_seed}_f{sfeature_idx}_steer.mp4', imgs)
Video(f'out/lowdim_seed{target_seed}_f{sfeature_idx}_steer.mp4', embed=True, width=256, height=256)

Seed 92400 Eval: 201it [00:19, 10.25it/s]                         

Reward for 92400:  0.9845608556213885
